In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from joblib import Parallel, delayed
from tqdm import tqdm
from itertools import product
from itertools import permutations
from itertools import combinations
from pyEDM import *
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.base import BaseEstimator, TransformerMixin
import time
import os
from copy import deepcopy
import math
import random
from sklearn.metrics import root_mean_squared_error
from scipy.stats import ttest_ind
import pickle
import ast
import json



from IPython.display import display, HTML
display(HTML('<style>.container { width:90% !important; }</style>'))

import warnings
warnings.filterwarnings("ignore", 
    message="A worker stopped while some jobs were given to the executor.",
    module="joblib.externals.loky.process_executor")

In [68]:
def get_block(data, num_lags=1, tau=1):
    ''' Get a dataframe with all the possible valid lags of the variables. '''
    
    block = pd.concat([data[var].shift(lag*tau).rename(f'{var}(t-{lag*tau})') for lag in range(num_lags+1) for var in data.columns], axis=1)

    return block
def get_xmap_results_smap(block, target, embeddings, Tp, theta, lib, pred):
    '''Function to do exhaustive search of embeddings.'''
    
    def compute_rho(block, target, embedding, Tp, theta, lib, pred):
        xmap = SMap(dataFrame=block, target=target, columns=embedding, Tp=Tp, theta=theta, embedded=True, lib=lib, pred=pred, noTime=True)
        rho = xmap['predictions'][['Observations', 'Predictions']].corr().iloc[0,1]
        return embedding, xmap['predictions'], rho

    xmap_results = pd.DataFrame(columns=['embedding', 'rho'])
    xmap_results = Parallel(n_jobs=-1)(delayed(compute_rho)(block, target, embedding, Tp, theta, lib, pred) for embedding in embeddings)
    xmap_results = pd.DataFrame(xmap_results, columns=['embedding', 'result', 'rho'])
    xmap_results = xmap_results.sort_values(by='rho', ascending=False).reset_index(drop=True)
    
    return xmap_results

def get_valid_lags_tau(block, target, tau, num_lags, system_variables):
    
    # Get lags of system variables
    system_variable_lags = []
    for var in system_variables:
        var_lags = [f'{var}(t{i})' if i < 0 else f'{var}(t-{i})' for i in range(num_lags * tau, 1)]
        var_lags = var_lags[::tau][:num_lags+1]
        system_variable_lags = system_variable_lags + var_lags
    
    # Remove (t-0) lag of target variable from valid_lags
    valid_lags = [x for x in system_variable_lags if x[-4:-1] != 't-0']

                    
    return valid_lags


def create_single_model(E,theta,target,i_cols,lib, pred,HAB_embed,showPlot=False):
    driver = f'{target}(t-0)'
    cols = i_cols + [driver]
    result = SMap(
        dataFrame = HAB_embed, 
        columns = cols,
        target = driver,
        lib = lib,  # Library from rows 0 to 700
        pred = pred,
        E = E+1,
        theta=theta,
        noTime=True,
        showPlot = showPlot,
        embedded=True,
        ignoreNan = True
    )
    return result

def thresh_bloom_binary_prediction(obs,pred,threshold=8.03199999999999):
    #obs_bloom_95 = np.percentile(obs, 95) #incorrect
    #pred_bloom_95 = np.percentile(pred, 95) #incorrect
    obs_blooms = obs > threshold
    pred_blooms = pred > threshold
    Accuracy = 1 - (obs_blooms ^ pred_blooms).mean()
    True_pos = (obs_blooms & pred_blooms).sum() / obs_blooms.sum()
    False_pos = ((~obs_blooms) & pred_blooms).sum() / (~obs_blooms).sum()
    True_neg = ((~obs_blooms) & (~pred_blooms)).sum() / (~obs_blooms).sum()
    False_neg = (obs_blooms & (~pred_blooms)).sum() / obs_blooms.sum()
    
    return [Accuracy, True_pos, False_pos, True_neg, False_neg]


def create_model(data,system_variables,target,Tp,num_lags,tau,theta_list,lib,pred):
    HAB_embed_block = get_block(data,50)
    valid_lags = get_valid_lags_tau(HAB_embed_block, target, tau, num_lags, system_variables)
    HAB_embeddings = {}
    for E in range(3,16):
        # Get random embeddings using valid lags
        embeddings = set()
        sample = 100
        max_trials = 100
        trials = 0
        while len(embeddings) < sample and trials < max_trials:
            embedding = tuple(random.sample(valid_lags, E))
            sorted_embedding = tuple(sorted(embedding))
            if sorted_embedding not in embeddings:
                embeddings.add(sorted_embedding)
            trials += 1
        embeddings = [list(embedding) for embedding in embeddings]
        HAB_embeddings['{0}'.format((target, E))] = embeddings
        E_list = range(4,16)


        
    total_iterations = len(E_list) * len(theta_list)

    parameters = pd.DataFrame(columns=['target', 'columns', 'E', 'theta', 'rho', 'rmse', 'pred'])

    with tqdm(total=total_iterations) as pbar:
        for E, theta in product(E_list, theta_list):

            key = [key for key in HAB_embeddings.keys() if eval(key)[0] == target and eval(key)[1] == E] #HAB_embeddings["('Avg_Chloro', 4, 0, 6)"][0]
            embeddings = HAB_embeddings[key[0]]
            for embedding in embeddings:
                smap_model = create_single_model(E,theta,target,embedding,lib, pred,HAB_embed_block,showPlot=False)
                df = smap_model['predictions']
                rho = df[['Observations', 'Predictions']].corr().iloc[0,1]
                rmse = root_mean_squared_error(df['Observations'].iloc[1:-1], df['Predictions'].iloc[1:-1])
                #bbp = thresh_bloom_binary_prediction(df['Observations'].iloc[1:-1],df['Predictions'].iloc[1:-1])

                new_row = {'target': target, 'columns': embedding + [f'{target} (t-0)'], 'E': E,'theta':theta, 'rho':rho, 'rmse':rmse, 'pred':df['Predictions']}
                parameters.loc[len(parameters)] = new_row

            

            pbar.update(1)
    parameters.sort_values(by='rho',ascending=False)
    return parameters

In [78]:
def ensemble_binary_bloom(parameters_df,n=300,p=0.05,samp=1,bloom_thresh=8.013):
    parameters_df = parameters_df.iloc[:n*samp].sample(n)
    sum = np.zeros(np.array(parameters_df['pred'].iloc[0][1:]).size)
    for i in range(n):
        curr = np.array(parameters_df['pred'].iloc[i][1:]) > bloom_thresh#np.percentile(parameters_df['pred'].iloc[i].iloc[1:],95)#
        sum = sum + curr
    return sum > (n*p)

'''
@parameters
data - dataframe of data containing column for target and desired system variables'
system variables - variables contained in system to be used for prediction
target - variable that will be forecasted (also used in prediction)
Tp - IDK
E - number of lags we go back
tau - step size that we go back at
theta_list - list of theta values to be used in ensemble model
lib - library used for prediction
pred - prediction length

@return
returns forecast for next time step given the dataframe 
'''

def next_forecast(data,system_variables,target,Tp,E,tau,theta_list,lib,pred,p=0.05):
    lib = '1 ' + str(data.shape[0] - 32) 
    pred = '' + str(data.shape[0] - 31) + ' ' + str(data.shape[0])
    parameters = create_model(data,system_variables,target,Tp,E,tau,theta_list,lib,pred)
    preds = ensemble_binary_bloom(parameters,n=300,p=0.05,samp=1,bloom_thresh=8.013)
    return preds
    


# Testings

In [79]:

paper_data = pd.read_csv('Data/data_w_gaps_and_wind.csv')
paper_data = paper_data.set_index('time')
paper_data['Time'] = paper_data.index.astype(int)
paper_data['Avg_Chloro'] #= paper_data['Avg_Chloro'].apply(np.log1p) #LOG AMPUTATION
#IMPUTE HAB DATA
#Build basic linear regression model as sanity check
# Custom impute missing values with the average of the value in front and behind of it 
class ForwardBackwardImputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_filled_forward = X.fillna(method='ffill').fillna(method='bfill')
        X_filled_backward = X.fillna(method='bfill').fillna(method='ffill')

        return (X_filled_forward + X_filled_backward) / 2


Imputer = ForwardBackwardImputer()
paper_data = paper_data.apply(pd.to_numeric, errors='coerce')
Imputer.fit(paper_data)
paper_data = Imputer.transform(paper_data)#COMMENT OUT IF DONT WANT MEAN MPUTE
paper_data
HAB_embed = get_block(paper_data,50)
paper_data

/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_47429/2944759716.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_filled_forward = X.fillna(method='ffill').fillna(method='bfill')
/var/folders/5y/74w_zv855875xvjtmhp4g1xw0000gn/T/ipykernel_47429/2944759716.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_filled_backward = X.fillna(method='bfill').fillna(method='ffill')


,Avg_Chloro,Nitrate,Phosphate,Silicate,Nitrite,SURF_TEMP_C,BOT_TEMP_C,SURF_SAL_PSU,BOT_SAL_PSU,SURF_DENS_kgm3,BOT_DENS_kgm3,AVG_TEMP_C,AVG_SAL_PSU,AVG_DENS_kgm3,WSPD,Time
time,,,,,,,,,,,,,,,,
0,1.82,0.43,0.24,5.6,0.03,20.3,15.9,33.77,33.62,1023.638094,1024.620655,18.10,33.695,1024.129374,4.691667,0.0
7,2.71,0.83,0.37,5.1,0.04,17.8,16.9,33.66,33.62,1024.186379,1024.391719,17.35,33.640,1024.289049,2.787500,7.0
13,0.99,0.45,0.24,3.5,0.00,22.2,18.3,33.74,33.58,1023.100557,1024.026040,20.25,33.660,1023.563298,3.412500,13.0
20,1.23,0.34,0.17,4.2,0.00,19.8,19.4,33.68,33.62,1023.701088,1023.780535,9.90,33.650,1023.740812,3.495833,20.0
28,2.41,0.29,0.12,3.8,0.00,22.5,22.5,33.73,33.69,1023.009067,1023.000725,11.25,33.710,1023.004896,2.604167,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,3.22,0.85,0.30,6.2,0.00,17.4,17.4,33.48,33.46,1024.146195,1024.152950,17.40,33.470,1024.149572,5.076389,5789.0
5796,3.39,0.45,0.25,4.9,0.00,17.9,17.9,33.48,33.47,1024.026335,1024.040616,17.90,33.475,1024.033476,2.737500,5796.0
5803,7.82,0.00,0.17,5.2,0.00,18.3,16.4,33.54,33.55,1023.974093,1024.454252,17.35,33.545,1024.214172,3.306250,5803.0


In [80]:
data = paper_data
system_variables = ['AVG_SAL_PSU', 'Avg_Chloro', 'Nitrate', 'SURF_TEMP_C','BOT_DENS_kgm3']
target = 'Avg_Chloro'
system_variables = system_variables
Tp = 1
E = 3 # up to -2 
tau = -1
theta_list = [1,5,9,15,25,35,45]
lib = '1 800'  # Library from rows 0 to 700
pred = '801 832'


preds = next_forecast(data,system_variables,target,Tp,E,tau,theta_list,lib,pred)
preds

 38%|███▊      | 32/84 [00:40<01:12,  1.39s/it]

In [ ]:
#Eval
for i in range(700,833):
    preds = next_forecast(data.iloc[],system_variables,target,Tp,E,tau,theta_list,lib,pred)
    